In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from habanero import Crossref

In [3]:
base_path = 'source/'
use_column = ['Article Title', 'Abstract', 'DOI']

In [4]:
def excel_list():
    folder = Path(base_path)
    return [f.name for f in folder.iterdir() if f.is_file()]

#e = excel_list()

In [5]:
def make_one(excel_list):

    concat_list = []

    def cutted_df(file_name):
        d = pd.read_excel(base_path + file_name)
        d = d[use_column]
        d = d.dropna(subset = use_column)
        return d
    
    for e in excel_list:
        concat_list.append(cutted_df(e))

    result = pd.concat(concat_list, axis=0, ignore_index=True)
    return result.drop_duplicates(subset=['DOI'])

#m = make_one(e)
#m.to_csv('make_one.csv', index=False)

In [6]:
def add_date(doi):
    try:
        paper = Crossref().works(ids = doi)['message']
        date_arr = paper['published-print']['date-parts'][0]
        return pd.Timestamp(year=date_arr[0], month=date_arr[1], day=1)
    except:
        return np.nan

def doi_to_date(file_name):
    df = pd.read_csv(file_name)
    df['Date'] = df['DOI'].apply(add_date)
    df = df.drop(columns = ['DOI'])
    df = df.dropna()
    return df

dt = doi_to_date('make_one.csv')
dt.to_csv('date_one.csv')